In [1]:
import numpy as np
from NonlinearController.mpc_utils import *
from NonlinearController.controllers import *
from NonlinearController.model_utils import *
from NonlinearController.lpv_embedding import *
from NonlinearController.systems import UnbalancedDisc, ReversedSinCosUnbalancedDisc, SinCosUnbalancedDisc
import deepSI
import qpsolvers as qp
import torch
import random
import time
from NonlinearController.utils import *

(CVXPY) Jul 20 01:18:13 PM: Encountered unexpected exception importing solver PROXQP:
ImportError('DLL load failed while importing instructionset: The specified module could not be found.')


In [2]:
qp.available_solvers

['cvxopt', 'cvxpy', 'ecos', 'osqp', 'quadprog', 'scs']

In [ ]:
from torch import nn
class sincos_output_net(nn.Module):
    def __init__(self, nx, ny, nu=-1, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh):
        super(sincos_output_net, self).__init__()
        from deepSI.utils import simple_res_net
        self.ny = tuple() if ny is None else ((ny,) if isinstance(ny,int) else ny)
        self.feedthrough = nu!=-1
        if self.feedthrough:
            self.nu = tuple() if nu is None else ((nu,) if isinstance(nu,int) else nu)
            net_in = nx + np.prod(self.nu, dtype=int)
        else:
            net_in = nx
        self.net = simple_res_net(n_in=net_in, n_out=np.prod((1,),dtype=int), n_nodes_per_layer=n_nodes_per_layer, \
            n_hidden_layers=n_hidden_layers, activation=activation)

    def forward(self, x, u=None):
        xu = x
        xu = self.net(xu).view(*((x.shape[0],)+(1,)))
        y = torch.cat([torch.sin(xu), torch.cos(xu)], dim=1)
        return y
    
# change input from sin cos to arctan2 of sin cos.
    #!!! There is probably a problem with the sin and cos being normalized at the arctan2
class arctan2_encoder_net(nn.Module):
    def __init__(self, nb, nu, na, ny, nx, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh):
        super(arctan2_encoder_net, self).__init__()
        from deepSI.utils import simple_res_net
        self.nu = tuple() if nu is None else ((nu,) if isinstance(nu,int) else nu)
        self.ny = tuple() if ny is None else ((ny,) if isinstance(ny,int) else ny)
        self.net = simple_res_net(n_in=nb*np.prod(self.nu,dtype=int) + na*np.prod((1,),dtype=int), \
            n_out=nx, n_nodes_per_layer=n_nodes_per_layer, n_hidden_layers=n_hidden_layers, activation=activation)

    def forward(self, upast, ypast):
        ypast = torch.mul(torch.atan2(ypast[:,:,0],ypast[:,:,1]),0.5)
        net_in = torch.cat([upast.view(upast.shape[0],-1),ypast.view(ypast.shape[0],-1)],axis=1)
        return self.net(net_in)

class SinCos_encoder(deepSI.fit_systems.SS_encoder_general):
    def __init__(self, nx=10, na=20, nb=20, na_right=0, nb_right=0, e_net_kwargs={}, f_net_kwargs={}, h_net_kwargs={}):
        super(SinCos_encoder, self).__init__(nx=nx, na=na, nb=nb, na_right=na_right, nb_right=nb_right, e_net_kwargs=e_net_kwargs, f_net_kwargs=f_net_kwargs, h_net_kwargs=h_net_kwargs)
        self.h_net = sincos_output_net
        # self.e_net = arctan2_encoder_net
        # self.f_net = sincos_state_net

    def init_nets(self, nu, ny): # a bit weird
        na_right = self.na_right if hasattr(self,'na_right') else 0
        nb_right = self.nb_right if hasattr(self,'nb_right') else 0
        self.encoder = self.e_net(nb=(self.nb+nb_right), nu=nu, na=(self.na+na_right), ny=ny, nx=self.nx, **self.e_net_kwargs)
        self.fn =      self.f_net(nx=self.nx, nu=nu,                                **self.f_net_kwargs)
        # self.hn =      nn.Identity(self.nx)
        self.hn =      self.h_net(nx=self.nx, ny=ny,                            **self.h_net_kwargs)

    def init_model(self, sys_data=None, nu=-1, ny=-1, device='cpu', auto_fit_norm=True, optimizer_kwargs={}, parameters_optimizer_kwargs={}, scheduler_kwargs={}):
        '''This function set the nu and ny, inits the network, moves parameters to device, initilizes optimizer and initilizes logging parameters'''
        if sys_data==None:
            assert nu!=-1 and ny!=-1, 'either sys_data or (nu and ny) should be provided'
            self.nu, self.ny = nu, ny
        else:
            self.nu, self.ny = sys_data.nu, sys_data.ny
            # if auto_fit_norm:
            #     self.norm.fit(sys_data)
            self.norm.ustd = 1.8
            self.norm.y0 = np.array([0.0,0.0])
            self.norm.ystd = np.array([1.0,1.0])
            print(self.norm.ystd)
        self.init_nets(self.nu, self.ny)
        self.to_device(device=device)
        parameters_and_optim = [{**item,**parameters_optimizer_kwargs.get(name,{})} for name,item in self.parameters_with_names.items()]
        self.optimizer = self.init_optimizer(parameters_and_optim, **optimizer_kwargs)
        self.scheduler = self.init_scheduler(**scheduler_kwargs)
        self.bestfit = float('inf')
        self.Loss_val, self.Loss_train, self.batch_id, self.time, self.epoch_id = np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
        self.init_model_done = True

In [ ]:
##################  System  #######################
dt = 0.1
# system = UnbalancedDisc(dt=dt, sigma_n=[0.014])
system = SinCosUnbalancedDisc(dt=dt, sigma_n=[0.0])
system.reset_state()

# system.x = system.f(system.x, 0.5)
# for i in range(500):
#     system.x = system.f(system.x, 0.0)

##################  MPC variable specification  #######################
# model = deepSI.load_system("NonlinearController/trained_models/unbalanced/ObserverUnbalancedDisk_dt01_nab_4_SNR_30_e250")
model = deepSI.load_system("NonlinearController/trained_models/sincos/sincosNet_dt0_1_e100_b1000_nf15_amp2_3_sn0_014")
Nc=10; nr_iterations = 1; nr_sim_steps = 100

w_min = -4; w_max = -w_min
q_min = [-1., -1.]; q_max = [1., 1.]
w0 = 0; q0 = [np.sin(0), np.cos(0)]

In [ ]:
# reference_theta = np.ones(nr_sim_steps+Nc)*(-1.)
reference_theta = randomLevelReference(nr_sim_steps+Nc, [25,30], [-2,2])
reference = np.vstack((np.sin(reference_theta),np.cos(reference_theta)))

In [ ]:
model.norm.y0 = np.array([0.0,0.0])
model.norm.ystd = np.array([1.0,1.0])

In [ ]:
##################  Offline Computation  #######################
nx = model.nx
nu = model.nu if model.nu is not None else 1
ny = model.ny if model.ny is not None else 1
nz = nx+ny
ne = 1

# initialize objective function matrices
P = np.eye(ny)*0.01
Q1 = np.zeros((ny,ny)); np.fill_diagonal(Q1, [50,50])
Q2 = np.zeros((nz,nz)); Q2[ny:,ny:] = np.eye(nx)
Omega1 = get_Omega(Nc, Q1)
Omega2 = get_Omega(Nc, Q2)

R = np.eye(nu)*1;# Q = np.zeros((nz,nz)); np.fill_diagonal(Q, [200.,200.,1.,1.]) #Q = np.matrix('10,0,0;0,10,0;0,0,10')# these are user defined
Psi = get_Psi(Nc, R)
# extended objective matrices for soft constraints
e_lambda = 100 # weighting of minimizing e in objective function
Ge = np.zeros((Nc*nu+ne,Nc*nu+ne)) 
Ge[-ne:,-ne:] = e_lambda

embedder = velocity_lpv_embedder_autograd(model, Nc, n_stages=20)

# normalize initial input and output
norm = model.norm
u0 = norm_input(w0, norm)
y0 = norm_output(q0, norm)

# initialize observer history input and output
nb = model.nb
uhist = torch.ones((1,nb))*u0
na = model.na
# yhist = torch.ones((1,na+1))*y0
yhist = torch.Tensor(y0[np.newaxis].T).repeat(1,na+1)[None,:,:]

# initial predicted states, input, and output
X_1 = np.tile(model.encoder(uhist,yhist).detach().numpy(),Nc+2).T
U_1 = np.ones((Nc+1)*nu)[np.newaxis].T*u0
# Y_1 = np.ones((Nc)*ny)[np.newaxis].T *y0
Y_1 = np.tile(y0[np.newaxis],Nc).T

# determine constraint matrices
u_min = norm_input(w_min, norm); u_max = norm_input(w_max, norm)
y_min = np.hstack((norm_output(q_min, norm), np.ones(nx)*-10000)); y_max = np.hstack((norm_output(q_max, norm), np.ones(nx)*10000)) # augmented with the velocity states
D, E, M, c = getDEMc(y_min, y_max, u_min, u_max, Nc, nz, nu)
Lambda = np.tril(np.ones((Nc,Nc)),0)

##################  Logging  #######################
log_q = np.zeros((ny,nr_sim_steps))
log_w = np.zeros((nu,nr_sim_steps))
log_e = np.zeros((ne,nr_sim_steps))
log_iterations = np.zeros((1,nr_sim_steps))
log_comp_t = np.zeros((4, nr_sim_steps*nr_iterations))

offset = 0

In [ ]:
##################  Online Computation  #######################

#++++++++++++++++++ start simulation step +++++++++++++++++++++++
for k in range(nr_sim_steps):
    # extend normalized reference to form of extended state
    
    r = extendReference((reference[:,k:k+Nc] - norm.y0[np.newaxis].T)/norm.ystd[np.newaxis].T, 0, ny, Nc)
    
    #++++++++++++++++++ start iteration +++++++++++++++++++++++
    for iteration in range(nr_iterations):
        component_start = time.time()
        
        # determine predicted velocity states and output
        dX0 = differenceVector(X_1[:-nx], nx)
        dU0 = differenceVector(U_1, nu)
        # determine extended state from predicted output and velocity states
        Z0 = extendState(Y_1, dX0, nx, ny, Nc)

        Z0 = extendState(Y_1, dX0, nx, ny, Nc)

        # determine lpv state space dependencies
        list_A, list_B, list_C = embedder(X_1, U_1)
        list_ext_A, list_ext_B, list_ext_C = extendABC(list_A, list_B, list_C, nx, ny, nu, Nc)

        # describe optimization problem
        c_terminal = np.vstack((r[-ny:,:],np.zeros((nx,1))))
        M_terminal = np.zeros((nx+ny,ny)); M_terminal[:ny,:] = np.eye(ny)
        E_terminal = np.zeros((nx+ny,nx)); E_terminal[ny:,:] = np.eye(nx)
        Sy = np.hstack((np.zeros((ny,(Nc-1)*ny)),np.eye(ny)))
        # Sz = np.hstack((np.zeros((nz,(Nc-1)*nz)),np.diag(np.hstack((np.zeros(ny),np.ones(nx))))))
        Sx = np.hstack((np.zeros((nx,(Nc-1)*nz+ny)),np.eye(nx)))
        Z = getZ(list_ext_C,Nc,ny,nz)
        Phi = get_Phi(list_ext_A, Nc, nz)
        Gamma = get_Gamma(list_ext_A, list_ext_B, Nc, nz, nu)

        Phi = get_Phi(list_ext_A, Nc, nz)
        Gamma = get_Gamma(list_ext_A, list_ext_B, Nc, nz, nu)
        G = 2*(Gamma.T @ (Z.T @ (Omega1 + Sy.T @ P @ Sy) @ Z + Omega2) @ Gamma + Psi)
        F = 2*(Gamma.T @ (Z.T @ (Omega1 @ (Z @ Phi @ Z0[:nz] - r) + Sy.T @ P @ (Sy @ Z @ Phi @ Z0[:nz] - r[-ny:])) + Omega2 @ Phi @ Z0[:nz]))
        # G = 2*(Psi + Gamma.T @ Omega @ Gamma)
        # F = 2*(Gamma.T @ Omega @ (Phi @ Z0[:nz] - r))

        # describe constraints
        L = (M @ Gamma + E @ Lambda)
        alpha = np.ones((Nc,1))*U_1[0,0]
        W = -(E @ alpha + (D + M @ Phi) @ Z0[:nz])

        # add soft constraints
        Ge[:Nc*nu, :Nc*nu] = G
        Fe = np.vstack((F, np.zeros((ne,1))))
        Le = np.hstack((L, -np.ones((Nc*2*(nz+nu)+2*nz,ne))))

        # add equality constraints
        A = (E_terminal @ Sx) @ Gamma
        Ae = np.hstack((A,np.zeros((nz,1))))
        b = c_terminal-(E_terminal @ Sx) @ Phi @ Z0[:nz]

        # solve for optimal U over prediction horizon
        opt_result = qp.solve_qp(Ge,Fe,Le,c+W,solver="osqp",initvals=np.hstack((dU0[:,0],0)))
        # opt_result = qp.solve_qp(Ge,Fe,solver="osqp",initvals=np.hstack((dU0[:,0],0)))
        # split optimization result in optimal input and soft bound variable e
        dU0[:,0] = opt_result[:Nc*nu]
        e = opt_result[-ne]
        
        # predict states
        Z1 = Phi @ Z0[:nz] + Gamma @ dU0
        # split extended state up into ouputs and velocity states
        Y0, dX1 = decodeState(Z1, nx, ny, Nc)
        # overwrite previous predicted states and output with new predicted states and output
        Y_1[2*ny:,0] = Y0[ny:-ny,0]; dX0[nx:,0] = dX1[:-nx,0] #change the shifting on the output to be consequential
        # save previous iteration of U_1
        U_1_old = np.copy(U_1)
        # determine new X_1 states from known x0 and predicted dX0
        for i in range(2,Nc+1):
            X_1[(i*nx):(i*nx+nx),:] = dX0[((i-1)*nx):((i-1)*nx+nx),:] + X_1[((i-1)*nx):((i-1)*nx+nx),:]
        for i in range(2,Nc+1):
            U_1[(i*nu):(i*nu+nu),:] = dU0[((i-1)*nu):((i-1)*nu+nu),:] + U_1[((i-1)*nu):((i-1)*nu+nu),:]

        log_comp_t[1, nr_iterations*k + iteration] = log_comp_t[1, nr_iterations*k + iteration] + time.time() - component_start

        # stopping condition
        if np.linalg.norm(U_1 - U_1_old) < 1e-2:
            break
    #++++++++++++++++++ end iteration +++++++++++++++++++++++
    
    # determine input from optimal velocity input
    u0 = dU0[:nu,0] + U_1[:nu,0]
    # denormalize input
    w0 = denorm_input(u0, norm)
    # measure output then apply input
    system.x = system.f(system.x, w0[0])
    q1 = system.h(system.x, w0[0])

    # normalize output
    y1 = norm_output(q1, norm)

    # shift history input and output for encoder
    for j in range(nb-1):
        uhist[0,j] = uhist[0,j+1]
    uhist[0,nb-1] = torch.Tensor(u0)
    for j in range(na):
        yhist[0,:,j] = yhist[0,:,j+1]
    yhist[0,:,na] = torch.Tensor(y1)
    # predict state with encoder
    x1 = model.encoder(uhist,yhist)

    # shift predicted states, input, and output one time step k; and replace with measured/observed values
    X_1[:-nx, :] = X_1[nx:, :]; X_1[-nx:, :] = X_1[-2*nx:-nx, :]; X_1[nx:2*nx, :] = x1.detach().numpy().T
    U_1[:-nu, :] = U_1[nu:, :]; U_1[-nu:, :] = U_1[-2*nu:-nu, :]; U_1[:nu, :] = u0
    Y_1[:-ny, :] = Y_1[ny:, :]; Y_1[-ny:, :] = Y_1[-2*ny:-ny, :]; Y_1[ny:2*ny, :] = y1[np.newaxis].T

    # log system signals
    log_q[:,k] = q1
    log_w[:,k] = w0
    log_e[:,k] = e
    log_iterations[:,k] = iteration+1
    
    # print progress
    print("Sim step: " + str(k) + ", iterations: " + str(iteration+1))
    
#++++++++++++++++++ end simulation step +++++++++++++++++++++++

In [ ]:
# logs from other scripts
# log_q_nmpc = np.load("NonlinearController/experiments/log_q.npy")
# log_w_nmpc = np.load("NonlinearController/experiments/log_w.npy")
# log_u_lpv = np.load("NonlinearController/experiments/u_log.npy")
# log_y_lpv = np.load("NonlinearController/experiments/y_log.npy")

# fig1 = plt.figure(figsize=[8.9, 8])
fig1 = plt.figure(figsize=[14, 11])

start_offset = 0

plt.subplot(3,1,1)
plt.plot(np.arange(nr_sim_steps-start_offset)*dt, log_w[0,start_offset:], label='velocity input')
# plt.plot(np.arange(nr_sim_steps-start_offset)*dt, np.hstack((log_u_lpv[1+start_offset:],log_u_lpv[-1])), label='lpv input')
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*w_max, 'r-.')#, label='max')
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*w_min, 'r-.')#, label='min')
# # plt.xlabel("time [s]")
plt.ylabel("voltage [V]")
plt.grid()
plt.legend(loc='lower right')

plt.subplot(3,1,2)
plt.plot(np.arange(nr_sim_steps-start_offset)*dt, log_q[0,start_offset:], label='velocity output')
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*q_max[0], 'r-.')#, label='max')
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*q_min[0], 'r-.')#, label='min')
# plt.plot(np.arange(nr_sim_steps-start_offset)*dt, np.hstack((log_y_lpv[2+start_offset:],log_y_lpv[-1])), label='lpv output')
plt.plot(np.arange(nr_sim_steps-start_offset)*dt, np.hstack((reference[0,1+start_offset:nr_sim_steps],reference[0,nr_sim_steps])), 'k--', label='reference', alpha=0.8, linewidth=1)
# plt.plot(np.arange(nr_sim_steps-5)*dt, log_q[0,4:-1] - log_q_nmpc[0,5:nr_sim_steps], label='vel-nmpc')
plt.xlabel("time [s]")
plt.ylabel("sin angle")
plt.grid()
plt.legend(loc='lower right')

plt.subplot(3,1,3)
plt.plot(np.arange(nr_sim_steps-start_offset)*dt, log_q[1,start_offset:], label='velocity output')
plt.plot(np.arange(nr_sim_steps-start_offset)*dt, np.hstack((reference[1,1+start_offset:nr_sim_steps],reference[1,nr_sim_steps])), 'k--', label='reference', alpha=0.8, linewidth=1)
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*q_max[0], 'r-.')#, label='max')
plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*q_min[0], 'r-.')#, label='min')
plt.xlabel("time [s]")
plt.ylabel("cos angle")
plt.grid()
plt.legend(loc='lower right')

# plt.subplot(2,2,3)
# plt.plot(np.arange(nr_sim_steps)*dt, log_e[0,:], label='e')
# plt.xlabel("time [s]")
# plt.ylabel("e")
# plt.grid()
# plt.legend(loc='upper right')

# plt.subplot(2,2,4)
# plt.plot(np.arange(nr_sim_steps)*dt, log_iterations[0,:], label='iQP')
# plt.plot(np.arange(nr_sim_steps)*dt, np.ones(nr_sim_steps)*nr_iterations, 'r-.', label='max iter')
# plt.xlabel("time [s]")
# plt.ylabel("iterations")
# plt.grid()
# plt.legend(loc='upper right')

# plt.savefig("Figures/offset_free_comparison.svg")
# plt.savefig("Figures/velocity_input_states.svg")
plt.show()

In [ ]:
CT_iters = np.split(log_comp_t, nr_sim_steps, axis=1)
CT = np.sum(CT_iters[0], axis=1)

remove_start = 0
S_iter = np.zeros(nr_sim_steps-remove_start)
T_iter = np.zeros(nr_sim_steps-remove_start)

for i in range(remove_start,nr_sim_steps):
    CT = np.sum(CT_iters[i], axis=1)
    S_iter[i-remove_start] = CT[1]
    T_iter[i-remove_start] = np.sum(CT)

Sorted = np.sort(T_iter)
# np.max(T_iter)*1000, np.mean(Sorted[int(nr_sim_steps*0.95):])*1000, np.mean(T_iter)*1000, np.std(T_iter)*1000, np.mean(S_iter)*1000 #in ms
np.max(T_iter)*1000,  np.mean(T_iter)*1000, np.std(T_iter)*1000, np.mean(S_iter)*1000 #in ms

In [ ]:
np.unique(log_iterations, return_counts=True)

In [ ]:
plt.plot(T_iter)

In [ ]:
fig2 = plt.figure(figsize=[8.9, 4.0])#['getAB', 'solve', 'overhead', 'sim']
data1 = np.trim_zeros(log_comp_t[0,remove_start*nr_iterations:])
data2 = np.trim_zeros(log_comp_t[1,remove_start*nr_iterations:])
data3 = np.trim_zeros(log_comp_t[2,remove_start*nr_iterations:])
data4 = np.trim_zeros(log_comp_t[3,remove_start*nr_iterations:])
data = [data1, data2, data3, data4]
plt.boxplot(data)
plt.xticks([1, 2, 3, 4],  ['getAB', 'solve', 'overhead', 'sim'])
plt.grid(axis='y')
plt.ylabel("time [s]")
plt.xlabel("components")
plt.show()